In [1]:
import json
import os
import requests

import dash_bootstrap_components as dbc
import numpy as np
import pandas as pd
import plotly.express as px
import xarray as xr

from dash import html
from dash import dcc
from pathlib import Path
from shapely.geometry import shape

import base64
import dash
import json
import os
import pprint
import tempfile

import dash_bootstrap_components as dbc
import layouts.mysleuth as sl
import numpy as np
import pandas as pd
import plotly.express as px
import rasterio as rio
import rasterio.warp as warp
import sleuth_sklearn.utils as utils

In [2]:
import numpy as np
import dash
from dash import html, dcc, callback, Input, Output
import dash_bootstrap_components as dbc

# Datos de ejemplo
id_hash = "example_id"
urban_rasters = np.random.rand(5, 100, 100)
years = [2016, 2017, 2018, 2019, 2020]

# Botones de idioma
language_buttons = dbc.ButtonGroup(
    [
        dbc.Button("Español", id="btn-lang-es", n_clicks=0),
        dbc.Button("English", id="btn-lang-en", n_clicks=0),
        dbc.Button("Portuguese", id="btn-lang-pt", n_clicks=0),
    ],
    style={"position": "absolute", "top": "10px", "right": "10px", "z-index": "1"},
)

# Inicializar la aplicación Dash
app = dash.Dash(__name__)

# Estructura básica de la aplicación
app.layout = html.Div([
    language_buttons,
    html.Div(id='content')  # Contenedor principal para contenido dinámico
])

# Función de callback para actualizar los contenidos
@app.callback(
    Output("content", "children"),
    [Input("btn-lang-es", "n_clicks"),
     Input("btn-lang-en", "n_clicks"),
     Input("btn-lang-pt", "n_clicks")]
)
def update_content(btn_lang_es, btn_lang_en, btn_lang_pt):
    ctx = dash.callback_context
    if not ctx.triggered:
        language = 'es'  # Idioma predeterminado
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]
        language = 'es' if button_id == 'btn-lang-es' else 'en' if button_id == 'btn-lang-en' else 'pt'

    # Llamada a la función summary con el idioma seleccionado
    summary_result = sl.summary(id_hash, urban_rasters, years, language=language)

    # Crear componentes Dash basados en los resultados de summary
    components = [
        html.H1("Traduccion"),
        html.P(f" "),
        html.Div(summary_result)  # Asumiendo que summary_result es adecuado para incluir directamente
    ]

    return components

if __name__ == '__main__':
    app.run_server(debug=True)


In [ ]:
def load_sleuth_predictions(path_cache, id_hash, mode):
    # Esta función debe cargar las predicciones de Sleuth
    pass

def calculate_coverage(worldcover, grids, start_year):
    # Esta función debe calcular la cobertura
    pass

def plot_sleuth_predictions(grids, start_year, num_years):
    # Esta función debe crear una figura con las predicciones de Sleuth
    pass

def plot_coverage(estimate_coverage, label):
    # Esta función debe crear una figura con la estimación de la cobertura
    pass

def summary(id_hash, urban_rasters, years, language='es'):


In [ ]:
def summary(id_hash, urban_rasters, years, language='es'):
    
    translations = {
        "es": {
        "year": "Año",
        "urbanization_percentage": "Porcentaje de Urbanización",
        "category": "Categoría",
        "observations": "Observaciones",
        "expansion": "Expansión",
        "general_summary": "Resumen General",
        "urban_area_increase": "+{0:.1f}% de aumento del área urbanizada 2070 vs. 2020.",
        "modes": ["inercial", "acelerada", "controlada"]
        },
        "en": {
        "year": "Year",
        "urbanization_percentage": "Urbanization Percentage",
        "category": "Category",
        "observations": "Observations",
        "expansion": "Expansion",
        "general_summary": "General Summary",
        "urban_area_increase": "+{0:.1f}% increase in urban area 2070 vs. 2020.",
        "modes": ["inertial", "accelerated", "controlled"]
        },
        "pt": {
        "year": "Ano",
        "urbanization_percentage": "Percentagem de Urbanização",
        "category": "Categoria",
        "observations": "Observações",
        "expansion": "Expansão",
        "general_summary": "Resumo Geral",
        "urban_area_increase": "+{0:.1f}% de aumento da área urbana 2070 vs. 2020.",
        "modes": ["inercial", "acelerada", "controlada"]
        }
    }

    path_cache = Path(f"./data/cache/{id_hash}")
    worldcover = np.random.rand(100, 100)  # Suponiendo que esta es la forma de obtener worldcover
    start_year = 2020
    num_years = 50

    id_hash = str(id_hash)
    historical_years = np.array(years)
    historical_grids = np.array(urban_rasters)
    
    modes_translated = translations[language]['modes']
    
    x = list(historical_years)
    y = [grid.sum() / grid.size for grid in historical_grids]
    z = [translations[language]['observations']] * len(x)
    final_y = y[-1]

    tabs = []
    coverage_graphs = []
    for mode in modes_translated:
        grids = load_sleuth_predictions(path_cache, id_hash, mode=mode)

        x_pred = list(range(start_year + 1, start_year + num_years + 1))
        y_pred = [grid.sum() / grid.size for grid in grids]
        z_pred = [f"{translations[language]['expansion']} {mode}"] * len(x_pred)

        x_pred = [start_year] + x_pred
        y_pred = [final_y] + y_pred

        x.extend(x_pred)
        y.extend(y_pred)
        z.extend(z_pred)

        # Plot Sleuth Predictions
        tab = dbc.Tab(
            dbc.Card(
                dbc.CardBody(
                    dbc.Container(
                        dbc.Row(
                            dbc.Col(
                                dcc.Graph(
                                    figure=plot_sleuth_predictions(grids, 2020, 50, language = language),
                                    responsive=True,
                                    style={"height": "60vh"},
                                ),
                                width=8,
                            )
                        )
                    )
                )
            ),
            label=f"{translations[language]['expansion']}: {mode}",
        )
        tabs.append(tab)

        # Coverage
        estimate_coverage = calculate_coverage(worldcover, grids, start_year)
        fig_coverage = plot_coverage(estimate_coverage, f"{translations[language]['expansion']}: {mode}", language = language)
        coverage_graphs.append(dcc.Graph(figure=fig_coverage))

    df = pd.DataFrame(
        zip(x, y, z), columns=[translations[language]['year'], translations[language]['urbanization_percentage'], translations[language]['category']]
    )
    fig = px.line(df, x=translations[language]['year'], y=translations[language]['urbanization_percentage'], color=translations[language]['category'])
    fig.update_yaxes(tickformat=",.0%")

    # Cambio Porcentual por Escenario
    base = df.loc[(df[translations[language]['year']] == 2020) & (df[translations[language]['category']] == translations[language]['observations'])][    translations[language]['urbanization_percentage']
    ].values[0]

    columns = []
    #for mode in modes_translated:
    for cat, color in zip(
        modes_translated, ["danger", "warning", "success"]
    ):
        
        cat = f"{translations[language]['expansion']} {mode}"#.capitalize()}"
        #prediction = df.loc[df[translations[language]['category']] == cat][translations[language]['urbanization_percentage']].values[0]
        prediction = df.loc[df[translations[language]['category']] == cat][translations[language]['urbanization_percentage']].values[0]

        percentage_increase = round(((prediction - base) / base) * 100, 1)
        col = dbc.Col(dbc.Card(
            dbc.CardBody(
                [
                    html.H5(cat, className="card-title"),
                    html.P(
                        f"+{percentage_increase}% {translations[language]['urban_area_increase'].format(percentage_increase)}",
                        className="card-text",
                    ),
                ]
            ),
            color=color,
            inverse=True,
        ))
        
        columns.append(col)

    cards = html.Div(dbc.Row(columns, className="mb-4"))
    all_elements = [cards] + coverage_graphs + [dcc.Graph(figure=fig)]
    plot_tab = dbc.Tab(dbc.Card(dbc.CardBody(all_elements)), label=translations[language]["general_summary"])
    tabs = [plot_tab] + tabs
    out = dbc.Tabs(tabs, active_tab="tab-0")

    return out